In [1]:
import numpy as np 
from scipy import misc
import matplotlib.pyplot as plt
import time
from mpl_toolkits import mplot3d
import plotly.graph_objects as go

### Rosenbrock functions ( Jacobian and Hessian), error function, linear research functions

In [ ]:
# Execute this cell before launching the other algorithms


def rosen(z):
    x,y=z
    return  (x-1)**2 + 100*(x**2-y)**2
                   
def diff_rosen(z):
    x,y=z
    return np.array([-400*x*(y-x**2)-2*(1-x),
       200*(y-x**2)])
        

def hess_rosen(z):
    x,y=z
    return np.array([
        [400*(-y+3*x**2)+2,  -400*x],
        [-400*x,    200]
    ])


#Error function

def compute_error(x,error):
    # We will measure the distance with the target point (1,1) at each iteration
    x_obj=np.array([1,1])
    return np.linalg.norm(np.array(x)-x_obj)<error


### GSS and Wolfe method

def section_doree(f,a,b,nmax):
    for i in range(nmax):
        to=(1+np.sqrt(5))/2
        a_temp=a+((b-a)/to**2)
        b_temp=a+((b-a)/to)
        if f(a_temp)<f(b_temp):
            a_1=a
            b_1=b_temp
        if  f(a_temp)>f(b_temp):
            a_1=a_temp
            b_1=b
        if  f(a_temp)==f(b_temp):
            a_1=a_temp
            b_1=b_temp
      
        a=a_1
        b=b_1
    return (a+b)/2

def wolfe(f,df,x,c1=1e-4, c2=0.9, max_iters=50):
    lr,lr_min,lr_max=1,0,0
    grad=df(x)
    d=-grad
    for i in range(max_iters):
        if (f(x + lr *d) <= f(x) + c1*lr*np.dot(grad,d)) and (f(x + lr *d) >= c2*np.dot(grad, d)):
            return lr
        else:
            if (f(x + lr *d) > f(x) + c1*lr*np.dot(grad,d)):
                lr_max=lr
            elif (f(x + lr *d) < c2*np.dot(grad, d)):
                lr_min=lr
        if lr_max==0:
            lr=2*lr_min
        else:
            lr=(lr_max-lr_min)/2
    return lr

### Vizualisation of the Rosenbrock function

In [ ]:
x, y = np.arange(-2,2.2,0.01), np.arange(-2,2.2,0.01)
X,Y=np.meshgrid(x,y)
z = rosen([X,Y])

surface_trace=go.Surface(z=z, x=x, y=y)

fig = go.Figure(data=[surface_trace])
fig.update_layout(title=' The Rosenbrock function',title_x=0.5)

### Newton's Method

In [ ]:
def newton2d(f,diff_rosen,z,epsilon=0.0000001):    
    nb_iteration=0
    liste_itere=[z]
    while True:        
        x,y = z        
        inv_H=np.linalg.inv(diff_rosen(z))
        x1,y1= np.array([x,y])-inv_H@f(z)
        liste_itere.append((x1,y1))
        z=x1,y1
        nb_iteration+=1
        
        if compute_error(z,epsilon):
            break          
    return liste_itere,z,nb_iteration 

init=[-1.9,2]
epsilon=0.0000001
start=time.time()
itéré,result,nb_iterations=newton2d(diff_rosen,hess_rosen,init)
end=time.time()

print(f"nb_iterations: {nb_iterations}, result: {result}\n")
print(f"{end-start} s needed to converge to a solution") 

### Gradient Descent with fixed learning rate

In [ ]:
def gradfixe(f,df,x,lr,epsilon,max_epochs):
    
    i=0
    while not compute_error(x,epsilon) and i<max_epochs:
        gradient=-df(x)
        x1=x+lr*gradient
        x=x1
        i+=1
    if i==max_epochs:
        print("maximum number of epoch reached, epsilon= ",np.linalg.norm(x-np.array([1,1])))
    return x1,i


x=np.array([-1.9,2])
error=0.0000001
lr=0.001
max_epochs=100000

start=time.time()
result,nb_iterations=gradfixe(rosen,diff_rosen,x,lr,error,max_epochs)
end=time.time()


print(f"\nnb_iterations: {nb_iterations}, result: {result}\n")

print(f"{end-start} s needed to converge to a solution") 


### Gradient descent with adaptative learning rate

In [ ]:
def gradopt(f,df,x,error,max_epochs,search_method):
    liste_iter=[x]
    i=0   
    while not compute_error(x,error) and i<max_epochs:
        gradient=df(x)
        if search_method=="section_doree":
            lr=section_doree(lambda lr:f(x-lr*gradient),0,1,50)
        elif search_method=="wolfe":
            lr=wolfe(f,df,x,0.3,0.7,1000)
        else:
            print(f"{seach_method} is not recognized")
            break
        x1=x-lr*gradient
        epsilon=np.linalg.norm(x-x1,2)
        i+=1
        x=x1
        liste_iter.append(x)
    if i==max_epochs:
        print("\nmaximum number of epoch reached, epsilon= ",np.linalg.norm(x-np.array([1,1])))
    return x1,liste_iter,i

x_init=np.array([-1.9,2])
error=0.0000001
max_epochs=100000
seach_method="wolfe" #either "section_doree" or "wolfe"

start=time.time()
result,liste_iter,nb_iterations=gradopt(rosen,diff_rosen,x_init,error,max_epochs,seach_method)
end=time.time()

print(f"\nnb_iterations: {nb_iterations}, result: {result}\n")
print(f"{end-start} s needed to converge to a solution") 

pts_x=np.array([liste_iter[i][0] for i in range(len(liste_iter))])
pts_y=np.array([liste_iter[i][1] for i in range(len(liste_iter))])
x, y = np.arange(-3,3,0.01), np.arange(-2,2.2,0.01)
X,Y=np.meshgrid(x,y)
z = rosen([X,Y])
line_trace = go.Scatter3d(x=pts_x,y=pts_y,z=rosen([pts_x,pts_y]), mode='lines', line=dict(color='red', width=4))
surface_trace=go.Surface(z=z, x=x, y=y)
fig = go.Figure(data=[surface_trace,line_trace])
fig.update_layout(title=' The Rosenbrock function and the steps achieved by the gradient descent before converging',title_x=0.5)
fig.show()

### BFGS/DFP

In [ ]:
def bfgs(f,df, x, max_iterations=1000, tolerance=1e-6,methos="BFGS"):
    B = np.eye(len(x))
    grad=df(x)   
    p=-grad
    liste_iter=[x]
    for i in range(max_iterations):
        lr=wolfe(f,df,x,0.3,0.7,50)
        x_old=x
        x=x_old+lr*p
        liste_iter.append(x)   
        if compute_error(x,tolerance):
            break
        grad_old=grad
        grad=df(x)
        s=x-x_old
        y=grad-grad_old
        y=y.reshape(2,1)
        s=s.reshape(2,1)
        r = 1/(y.T@s)
        #We try to approximate directly the inverse of the Hessian in both BFGS and DFP
        if method=="BFGS":
            li = np.eye(2)-(r*(s@y.T))
            ri = np.eye(2)-(r*(y@s.T))
            hess_inter = li@B@ri
            B = hess_inter + (r*(s@s.T))    
        elif method=="DFP":     
            B=B+(s@s.T/(s.T@y))-(B@y@y.T@B/(y.T@B@y))   
        else:
            print(f"{method} is not recognized")
            break
        p = -np.dot(B, grad)
    if i==max_iterations-1:
        print("\nmaximum number of epoch reached, epsilon= ",np.linalg.norm(x-np.array([1,1])))
    return x,liste_iter,i

x_init=np.array([-1.9,2])
error=0.0000001
max_iter=1000000
method="DFP" # either "BFGS" or "DFP"

start=time.time()
result,liste_iter,nb_iterations=bfgs(rosen,diff_rosen,x_init,max_iter,error,method)
end=time.time()

print(f"\nnb_iterations: {nb_iterations}, result: {result}\n")

print(f"{end-start} s needed to converge to a solution") 

pts_x=np.array([liste_iter[i][0] for i in range(len(liste_iter))])
pts_y=np.array([liste_iter[i][1] for i in range(len(liste_iter))])
x, y = np.arange(-3,3,0.01), np.arange(-2,2.2,0.01)
X,Y=np.meshgrid(x,y)
z = rosen([X,Y])

line_trace = go.Scatter3d(x=pts_x,y=pts_y,z=rosen([pts_x,pts_y]), mode='lines', line=dict(color='red', width=4))
surface_trace=go.Surface(z=z, x=x, y=y)

fig = go.Figure(data=[surface_trace,line_trace])
fig.update_layout(title=f' The Rosenbrock function and the steps achieved using {method} before converging',title_x=0.5)

fig.show()

## Conjugate gradient

In [ ]:
# Creating H and b
def create_matrixes(n):
    matrix = np.zeros((n, n)) 
    np.fill_diagonal(matrix, 4)
    for i in range(n-1):
        matrix[i, i+1] = -2
        matrix[i+1, i] = -2
        
    return matrix,np.ones(n)

In [ ]:
def conjGrad(H,x,b,tolerance,N): #finding x so that Hx=b
    
    r = b - H@x
    
    p = r.copy()
    for i in range(N):
        Hp = H@p
        alpha = p@r/(p@Hp)
        x = x + alpha*p
        r = b - H@x
        if np.sqrt(np.sum(r**2)) < tolerance:
            break
        else:
            beta = -r@Hp/(p@Hp)
            p = r + beta*p
    return x 

n=100
x=5*np.random.rand(n) #random x
error=0.0000001

start=time.time()
H,b=create_matrixes(n)
end=time.time()

result=list(np.around(conjGrad(H,x,b,error,n),5))

print(f"Convergence achieved. Results:{result}")

print(f"{end-start} s needed to converge to the solution") 